# Основы Python. Часть 3.2

ООП. Классы. Инкапсуляция. Наследование. Полиморфизм.

## Ограничения функций

### Пример 1

Задача. Надо уметь писать в файл в любом месте программы.

In [1]:
def write(filename, some_string):
    
    with open(filename, 'a') as f:

        f.write(some_string)  # пишем строку в файл

write('/tmp/f.txt', 'Hi!\n')

# ... (много кода)

write('/tmp/f.txt', 'Hi!\n')

Было бы хорошо задавать файл журнала один раз.

In [2]:
filename = '/tmp/f.txt'

write(filename, 'Hi!\n')

# ... (много кода)

write(filename, 'Hi!\n')

Может захотется, чтобы создавался новый файл, после достижения определённого размера.

In [3]:
import os

file_index = 0

def write(filename, some_string):
    
    MAX_SIZE = 20
    
    global file_index
    
    if os.path.isfile(filename):
    
        file_size = os.path.getsize(filename)

        if file_size > MAX_SIZE:  # если размер больше установленного значения, увеличиваем индекс файла на 1                

            file_index += 1                        
    
    with open(filename, 'a') as f:

        f.write(some_string)  # пишем строку в файл

In [4]:
write('/tmp/f_{}.txt'.format(file_index), 'Hi!\n')

# ...

write('/tmp/f_{}.txt'.format(file_index), 'Hi!\n')

### Пример 2

Задача. Функция, добавляющая к каждому элементу текущего списка последнее значение предыдущего.

In [5]:
last_element = None  # для хранения 

def process(some_list):
    
    global last_element  # говорим явно, что это переменная берётся из глобальной области видимости.
    
    if last_element is None:  # если не инициализированно
        
        last_element = some_list[-1]  # то сохраняем последний элемент текущего списка
        
    else:  # если уже есть значение
        
        some_list[:] = map(lambda x: x + last_element, some_list)  # добавляем это значение к каждому элементу списка

In [6]:
l = [1, 2, 3]

process(l)

print(l)

process(l)

print(l)

process(l)

print(l)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


## Инкапсуляция

Хотим скрыть реализацию записи в файл.

In [7]:
class Logger:
    def __init__(self, name, file='/tmp/log'):  # конструктор, вызывается при создании объекта
        self.__name = name  # сохраняем название логгера в приватную переменную __name
        self.__file = file  # сохраняем имя файла в приватную переменную __file

    def write(self, text):  # публичный метод, который пишет строку в файл
        with open(self.__file, 'a') as f:
            f.write(text)
            
    def __str__(self):  # переопределение встроенного метода класса __str__ для удобного вывода информации об объкете
        return self.__name

logger = Logger('log1')  # создаём объект класса Logger

In [8]:
print(logger)

log1


In [9]:
# можно посмотреть на интерфейс класса logger
dir(logger)

['_Logger__file',
 '_Logger__name',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'write']

In [10]:
# т.к. мы объявили переменную __filename с двумя "__", то это значит, что она не будет доступна извне этого класса.
logger.__filename

AttributeError: 'Logger' object has no attribute '__filename'

In [11]:
logger.write('Hi!')
logger.write('Hi!')
logger.write('Hi!')

Мы можем захотеть, чтобы создавался новый файл, после достижения определённого размера файла.

In [12]:
import os

class Logger:
    __MAX_SIZE = 30  # in bytes
    
    def __init__(self, name, file='/tmp/log'):
        self.__name = name
        self.__file = file
        self.__file_index = 0
        
    def __get_filename(self):
        return '{}_{}'.format(self.__file, self.__file_index)

    def write(self, text):
        with open(self.__get_filename(), 'a') as f:  # пишем в файл
            f.write(text)
            
        file_size = os.path.getsize(self.__get_filename())  # определяем размер файла
        
        if file_size > self.__MAX_SIZE:  # если размер больше установленного значения, увеличиваем индекс файла на 1
            self.__file_index += 1                                
            
    def __str__(self):
        return self.__name

In [13]:
logger = Logger('log1')

In [14]:
logger.write('Hello, world!\n')

## Наследование

In [15]:
class BaseLogger:
    """Base logger class"""
    
    def __init__(self, name, *args, **kwargs):
        """base constructor"""
        self.__name = name
    
    def write(self, text):
        """writes text to log"""
        
    def __str__(self):
        """return logger name"""
        return self.__name
        
        
class FileLogger(BaseLogger):
    """File logger writes text to file"""
    
    def __init__(self, *args, **kwargs):
        
        super(FileLogger, self).__init__(*args, **kwargs)
        
        self.__file = kwargs.get('file', None)
        
        if self.__file is None:
            
            raise Exception('You must enter file.')
        
    def write(self, text):
        
        super(FileLogger, self).write(text)
        
        with open(self.__file, 'a') as f:
            
            f.write(text)  
            
        print('Write {} to file {}'.format(text, self.__file))

            
class URLLogger(BaseLogger):
    """URL logger sends text to specified URL"""
    
    def __init__(self, *args, **kwargs):        
        
        super(URLLogger, self).__init__(*args, **kwargs)        
        
        self.__url = kwargs.get('url', None)
        
        if self.__url is None:
            
            raise Exception('You must enter URL.')  
            
    def write(self, text):
        
        super(URLLogger, self).write(text)
        
        # implementation is skipped
        print('Send {} to {}'.format(text, self.__url))

In [16]:
f_logger = FileLogger('f_logger', file='/tmp/f.txt')
u_logger = URLLogger('u_logger', url='http://someurl')

In [17]:
f_logger.write('Hi!')

Write Hi! to file /tmp/f.txt


In [18]:
u_logger.write('Hi!')

Send Hi! to http://someurl


### Наследование методов и переменных класса

Публичная переменная (PUBLIC) доступна внутри методов класса насследника и снаружи.

In [19]:
class A:
    name = 'name'
    
class B(A):
    def __init__(self):
        print('%s in child class' % self.name)

b = B()

b.name

name in child class


'name'

Защищённая переменная (PROTECTED) доступна внутри методов класса наследника, и доступна снаружи. 

Некоторые автодополнители не дают защищённую переменную в область видимости снаружи.

В некоторых StyleGuide's есть рекомендации, что такие переменные должны интерпретироваться как защищённые/protected. То есть доступные только методам класса наследника и недоступны снаружи.

In [20]:
class A:
    _name = 'name'
    
class B(A):
    def __init__(self):
        print('%s in child class' % self._name)

b = B()

b._name

name in child class


'name'

Закрытая переменная (PRIVATE) доступна только методам данного класса.

In [21]:
class A:
    __name = 'name'
    
class B(A):
    def __init__(self):
        print('%s in child class' % self.__name)

b = B()

b.__name

AttributeError: 'B' object has no attribute '_B__name'

### Наследование от нескольких классов

In [22]:
class A:
    def __init__(self):
        super().__init__() 
        print('A init')
        
class B:
    def __init__(self):
        super().__init__() 
        print('B init')
        
class C(A, B):
    def __init__(self):
        super().__init__()
        print('C init')
        
c = C()       

B init
A init
C init


In [23]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

### Абстрактные классы и методы

In [24]:
from abc import ABC, abstractmethod

class A(ABC):
    
    @abstractmethod
    def f(self):
        """Мы хотим, чтобы те классы, которые унаследовались, обязательно переопределили этот метод"""

In [25]:
a = A()

TypeError: Can't instantiate abstract class A with abstract methods f

In [26]:
class B(A):
    pass

b = B()

TypeError: Can't instantiate abstract class B with abstract methods f

In [27]:
class C(A):
    def f(self):
        pass
    
c = C() # Ok

## Полиморфизм

Базовый класс имеет общий интерфейс. Но реализация каждой конкретной функции выбирается в зависимости от конкретного класса, который унаследовался от базового.

In [28]:
loggers = []

for i in range(5):
    loggers.append(FileLogger('f_logger_{}'.format(i), file='/tmp/f_{}.txt'.format(i)))
    
for i in range(5):
    loggers.append(URLLogger('u_logger_{}'.format(i), url='http://someurl'))

In [29]:
for logger in loggers:
    print(logger)

f_logger_0
f_logger_1
f_logger_2
f_logger_3
f_logger_4
u_logger_0
u_logger_1
u_logger_2
u_logger_3
u_logger_4


In [30]:
for logger in loggers:
    logger.write('Hello!')

Write Hello! to file /tmp/f_0.txt
Write Hello! to file /tmp/f_1.txt
Write Hello! to file /tmp/f_2.txt
Write Hello! to file /tmp/f_3.txt
Write Hello! to file /tmp/f_4.txt
Send Hello! to http://someurl
Send Hello! to http://someurl
Send Hello! to http://someurl
Send Hello! to http://someurl
Send Hello! to http://someurl


## Методы

Объявление класса

In [31]:
class A:
    pass

Создание объекта класса A

In [32]:
a = A()

Объявление класса с методами

In [33]:
class A:
        
    """переменная, инициализированная при создании объявления класса"""
    a = 1  
    
    def __init__(self, *args, **kwargs):
        """Встроенный метод. Конструктор. Вызывается при создании объекта"""
        
    def method(self, a, b):
        """Публичный метод объекта класса A"""
        
    def _method(self, a, b):
        """Защищённый метод класса A"""
        
    def __method(self, a, b):
        """Закрытый метод класса А"""
    
    @staticmethod
    def static_method(a, b):
        """Статический метод класса. 
        Может быть вызван без создания объекта класса А.
        Не может читать или модифицировать аттрибуты объекта и самого класса.
        """
        
    @classmethod
    def class_method(cls, a, b):
        """Метод класса. Доступно окружение класса. 
        Может быть вызван без создания объекта класса А.
        Имеет и доступ и может модифицировать атрибуты класса, доступные всем объектам.
        """   
        
    # + @abstractmethod (будет рассказано позже)
    

In [34]:
A.static_method(1, 2)  # не знает о переменной 'a' класса

A.class_method(1, 2)  # знает о переменной 'a' класса

In [35]:
class A:
    b = 1
    
    @classmethod
    def set_b(cls, new_b):
        cls.b = new_b        

In [36]:
a = A()
a.b

1

In [37]:
b = A()
b.b

1

In [40]:
A.set_b(2)

In [41]:
a.b

2

In [42]:
b.b

2

### Встроенные методы

## `__init__`

In [43]:
class A:
    def __init__(self):
        """Конструктор. Вызывается при создании объекта класса"""
        print('Я только что создан')
        
a = A()

Я только что создан


## `__str__`

In [44]:
class A:
    def __init__(self, name):
        self.__name = name
        
    def __str__(self):
        """Строковое представление объекта"""        
        return self.__name
        
a = A('Name')

print(a)

Name


## `__repr__`

In [45]:
class A:
    def __init__(self, name):
        self.__name = name
        
    def __repr__(self):
        return 'A <{}>'.format(self.__name)
    
l = [A('one'), A('two'), A('three')]

l

[A <one>, A <two>, A <three>]

## `__len__`

In [46]:
class A:
    def __init__(self, _list):
        self.__list = _list
        
    def __len__(self):
        return len(self.__list)
    
a = A([1, 2, 3])

len(a)

3

## `__eq__` и другие методы сравнения

In [47]:
class A:
    def __init__(self, name):
        self.__name = name
        
    def __eq__(self, other):
        return self.__name == other.__name
    
a_1 = A('name')
a_2 = A('name')

a_1 == a_2

True

## `__next__`

In [48]:
class Iterator:
    def __init__(self):
        self.__i = 0
        
    def __iter__ (self):
        return self        
        
    def __next__(self):
        self.__i += 1
        return self.__i

In [49]:
it = Iterator()

In [50]:
next(it)

1

## `__bool__`

In [51]:
class A:
    def __bool__(self):
        return False
    
a = A()

bool(a)

False

## `__enter__` && `__exit__`

In [52]:
class Writer:
    def __init__(self, filename):
        self.__filename = filename
        self.__file = None
        
    def __enter__(self):
        self.__file = open(self.__filename, 'w')
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.__file.close()
        
    def write(self, text):
        self.__file.write(text)

In [53]:
with Writer('/tmp/f') as w:
    w.write('text')
    
# здесь файл уже закрыт

## `__add__` и другие арифметические и битовые специальные методы

In [54]:
class A:
    def __init__(self, _list):
        self.__list = _list
        
    def __add__(self, other):
        _res_list = [l + o for l, o in zip(self.__list, other.__list)]
        return A(_res_list)
    
    def __repr__(self):
        return ', '.join([str(_) for _ in self.__list])
    
a = A([1, 2, 3])

b = A([4, 5, 6])

a + b

5, 7, 9

### `__contains__`

In [55]:
class A:
    def __init__(self, _list):
        self.__list = _list
        
    def __contains__(self, item):
        return item in self.__list
    
a = A([1, 2, 3])

1 in a

True

Встроенных методов, которые можно переопределить очень много

https://docs.python.org/3.5/reference/datamodel.html